# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [76]:
import pandas as pd #Импортитруем библиотку Pandas
from pymystem3 import Mystem  #Импорт библиотеки с функцией лемматизации на русском языке — pymystem3
m = Mystem()

In [77]:
data = pd.read_csv('/datasets/data.csv') #Сохраняем файл в переменной
data.info() #Изучаем общую информацию

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Итак, в таблице 12 столбцов, с различными типами данных.
Согласно документации к данным:
* children — количество детей в семье
* days_employed — трудовой стаж в днях
* dob_days — возраст клиента в годах
* education — образование клиента
* education_id — идентификатор образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — доход в месяц
* purpose — цель получения кредита
 
В каждой строке таблицы содержатся данные о клиенте банка. Часть колонок хранит категориальные типы данных, описывая клиента больше как ячейку общества. Другая часть колонок содержит количественные  типы, которые уже больше характеризуют человека именно как клиента банка. В таблице также присутствуют столбцы-идентификаторы, что несколько облегачает обработку данных. 

**Вывод**

Предварительно можно утверждать, что, данных достаточно для проверки гипотезы. 
В столбцах "days_employed" и "total_income" количество значений отличается от остальных столбцов, но равно между собой. Скорее всего, существует взаимосвязь - почему в этих столбцах есть пропущенные значения и их количество равно. 

Чтобы двигаться дальше, нужно разобраться с пропущенными значениями в данных.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [78]:
data.isna().sum() #Посчитаем количество пропущенных значений


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропуски существуют , как мы уже заметили, в двух столбцах. Выведем часть таблицы на экран и посмотрим какого типа пропуск(и) нам встретятся.

In [79]:
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


В строке с индексом 12 есть пропуски типа NaN. Тип занятости для этого клиента указан "пенсионер". Мало вероятно что человек за всю жизнь не рабтал ни дня, скорее всего данные утеряны. 
Помимо пропусков в столбце обнаружились и другие проблемы: 
* почти все значения в столбце являются отрицательными(чего не может быть со стажем); 
* положительные значения сильно отличаются от отрицательных по модулю(в переводе на годы стаж будет равен около 1000 лет); 
* все значения являются дробными(что не очень удобно). 

In [80]:
incorrect_days_employed_counter = len(data[data['days_employed'] > 0])
print('Количество строк с некорректным значением стажа:', incorrect_days_employed_counter)

Количество строк с некорректным значением стажа: 3445


Количество строк с некорректным стажем от общего числа составляет примерно 16%. Примечательно что `тип занятости` в этих строках чаще  указан как "пенсионер". Можно предположить что годами ранее принято было записывать стаж в часах, 
поэтому разделим этот стаж на количество часов в сутках.

In [81]:
data_abs = data['days_employed'].abs() #Переведем все значения в положительные
data['days_employed'] = data_abs / 365 #Переведем значения дней в годы для удобства
data 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23.116912,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,11.026860,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,15.406637,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,11.300677,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,932.235814,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,12.409087,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,942.294258,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,5.789991,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,8.527347,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [82]:
#Переведем тысячи лет стажа во вразумительные цифры с помощью функции
def years_employed(row):
    days_employed = row['days_employed']
    dob_years = row['dob_years']
    if days_employed > dob_years: #Сравним значение стажа с возрастом клиента
        return days_employed / 24
    else:
        return days_employed
data['days_employed'] = data.apply(years_employed, axis=1)    
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23.116912,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,11.026860,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,15.406637,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,11.300677,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,38.843159,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,12.409087,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,39.262261,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,5.789991,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,8.527347,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


Около 2-х тысяч пропусков на 21525 строк это примерно 10%. Довольно весомое количество, которое лучше обработать и заменить на медианное значение стажа и медианное значение заработной платы. 

Заменим пропуски, используя группировку с двумя переменными. Такой метод должен дать более точные результат. Группировку по стажу будем проводить используя колонки `Семейное положение` и `Пол`. 

In [83]:
medians = (data.groupby(['family_status', 'gender'])
          .agg({'days_employed':'median'}).rename(columns = {'days_employed':'median_days_employed'}))
data = data.merge(medians, on = ['family_status', 'gender'])
data[['family_status', 'gender', 'days_employed', 'median_days_employed']][data['days_employed'].isna()] #оценим визуально 
#разницу в стаже


,family_status,gender,days_employed,median_days_employed
12,женат / замужем,F,NaN,6.962675
20,женат / замужем,F,NaN,6.962675
24,женат / замужем,F,NaN,6.962675
27,женат / замужем,F,NaN,6.962675
30,женат / замужем,F,NaN,6.962675
...,...,...,...,...
21488,вдовец / вдова,M,NaN,9.162563
21503,вдовец / вдова,M,NaN,9.162563
21505,вдовец / вдова,M,NaN,9.162563
21506,вдовец / вдова,M,NaN,9.162563


In [84]:
data.loc[data['days_employed'].isna(), 'days_employed'] = data.loc[data['days_employed'].isna(), 'median_days_employed'] 
#заменяем пропуски стажа

In [85]:
#проделываем то же самое с пропусками в колонке Доходы, группируя данные по колонкам Образование и Семейное положение
medians = (data.groupby(['education', 'family_status'])
          .agg({'total_income':'median'}).rename(columns =  {'total_income':'median_total_income'}))
data = data.merge(medians, on = ['education', 'family_status'])
data[['education', 'family_status', 'total_income', 'median_total_income']][data['total_income'].isna()]

,education,family_status,total_income,median_total_income
3,высшее,женат / замужем,NaN,178030.715657
4,высшее,женат / замужем,NaN,178030.715657
5,высшее,женат / замужем,NaN,178030.715657
7,высшее,женат / замужем,NaN,178030.715657
11,высшее,женат / замужем,NaN,178030.715657
...,...,...,...,...
21507,начальное,Не женат / не замужем,NaN,116656.560575
21509,начальное,Не женат / не замужем,NaN,116656.560575
21514,НЕОКОНЧЕННОЕ ВЫСШЕЕ,Не женат / не замужем,NaN,122584.409247
21515,НЕОКОНЧЕННОЕ ВЫСШЕЕ,Не женат / не замужем,NaN,122584.409247


In [86]:
data.loc[data['total_income'].isna(), 'total_income'] = data.loc[data['total_income'].isna(), 'median_total_income'] #заменяем
data.isna().sum() #проверяем

children                0
days_employed           0
dob_years               0
education               0
education_id            0
family_status           0
family_status_id        0
gender                  0
income_type             0
debt                    0
total_income            0
purpose                 0
median_days_employed    0
median_total_income     0
dtype: int64

**Вывод**

Мы определили количество пропущенных значений и их тип. Пропуски NaN относятся к числовому типу.

Избавились от пропусков заменяя значения медианным числом, используя группировку с двумя переменными, для более точного  результата. С помощью цикла и функции мы определили количество значений с невероятно-большим стажем и заменили его адекватным числом. В данном случае удаление или игнорирование пустых и некорректных значений может привести к неправильному исходу, т.к их количество больше 10% от всего объема данных.


### Замена типа данных

In [87]:
data['days_employed'] = data['days_employed'].astype('int') #Заменим тип данных float на тип int
data['total_income'] = data['total_income'].astype('int') 
data.info() #проверим

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 0 to 21524
Data columns (total 14 columns):
children                21525 non-null int64
days_employed           21525 non-null int64
dob_years               21525 non-null int64
education               21525 non-null object
education_id            21525 non-null int64
family_status           21525 non-null object
family_status_id        21525 non-null int64
gender                  21525 non-null object
income_type             21525 non-null object
debt                    21525 non-null int64
total_income            21525 non-null int64
purpose                 21525 non-null object
median_days_employed    21525 non-null float64
median_total_income     21525 non-null float64
dtypes: float64(2), int64(7), object(5)
memory usage: 2.5+ MB


**Вывод**

Методом astype() мы заменили вещественный тип на количественный везде, где это было необходимо. С измененным типом будет удобнее и визуально комфортнее работать.

### Обработка дубликатов

Для начала поищем в таблице явные дубликаты.

In [88]:
print('Количество явных дубликтов в таблице:', data.duplicated().sum())

Количество явных дубликтов в таблице: 54


In [89]:
data = data.drop_duplicates().reset_index(drop=True) #удалим явные дубликаты, удляя старые индексы и добавляя новые


При работе с таблицей были замечены различные варианты записи колонки `Образование`. Обработаем эту колонку.

In [90]:
data['education'].sort_values().unique() #вызовем уникальные значения

array(['ВЫСШЕЕ', 'Высшее', 'НАЧАЛЬНОЕ', 'НЕОКОНЧЕННОЕ ВЫСШЕЕ',
       'Начальное', 'Неоконченное высшее', 'СРЕДНЕЕ', 'Среднее',
       'УЧЕНАЯ СТЕПЕНЬ', 'Ученая степень', 'высшее', 'начальное',
       'неоконченное высшее', 'среднее', 'ученая степень'], dtype=object)

In [91]:
# Функция для замены неявных дубликатов
def replace_wrong_educations(wrong_educations, correct_education):  # на вход функции подаются список неправильных значений и строка с правильным значением
    for wrong_education in wrong_educations:  # перебираем неправильные имена
        data['education'] = data['education'].replace(wrong_educations, correct_education) # и для каждого неправильного имени вызываем метод replace()
replace_wrong_educations(['ВЫСШЕЕ', 'высшее'], 'Высшее') # Устранение неявных дубликатов
replace_wrong_educations(['НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'неоконченное высшее'], 'Неоконченное высшее')
replace_wrong_educations(['УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'], 'Ученая степень')  
replace_wrong_educations(['СРЕДНЕЕ', 'среднее'], 'Среднее')  
replace_wrong_educations(['НАЧАЛЬНОЕ', 'начальное'], 'Начальное')  
data['education'].sort_values().unique() #проверяем 

array(['Высшее', 'Начальное', 'Неоконченное высшее', 'Среднее',
       'Ученая степень'], dtype=object)

In [92]:
print('Количество дубликатов после изменений в столбце `Образование`:', data.duplicated().sum())

Количество дубликатов после изменений в столбце `Образование`: 0


**Вывод**

При работе с данной таблицей было обнаружено 54 явных дубликата. 

Явные дубликаты могли появиться из-за замены пустых значений на медианные. Мы их удаляем, т.к дубликаты могут исказить окончательные результаты.

Неявные дубликаты не были обнаружены.

### Лемматизация

Проведем лемматизацию столбца `Цель кредита`. Полученный результат пригодится для дальнейшей работы.

In [93]:
def lemm_group(purpose): #проведем лемматизацию колонки `Цель кредита` с помощью функции
    for word in purpose.split():
        lemmas = m.lemmatize(purpose) 
        if  'свадьба' in lemmas:         
            return "свадьба"           
        if 'недвижимость' in lemmas: 
            return 'недвижимость'
        if  'жилье' in lemmas:      
            return 'недвижимость'
        if 'автомобиль' in lemmas:
            return 'автомобиль'
        if 'образование' in lemmas:
            return 'образование'
        return "другое"              
data['purpose_group'] = data['purpose'].apply(lemm_group) #создадим отдельную колонку
data.head(10) #проверяем

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_days_employed,median_total_income,purpose_group
0,1,23,42,Высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,6.962675,178030.715657,недвижимость
1,0,7,43,Высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,6.962675,178030.715657,недвижимость
2,1,1,26,Высшее,0,женат / замужем,0,F,сотрудник,0,187863,строительство собственной недвижимости,6.962675,178030.715657,недвижимость
3,0,6,52,Высшее,0,женат / замужем,0,F,пенсионер,0,178030,покупка жилья для семьи,6.962675,178030.715657,недвижимость
4,2,6,50,Высшее,0,женат / замужем,0,F,сотрудник,0,178030,жилье,6.962675,178030.715657,недвижимость
5,2,6,35,Высшее,0,женат / замужем,0,F,сотрудник,0,178030,операции с жильем,6.962675,178030.715657,недвижимость
6,0,1,51,Высшее,0,женат / замужем,0,F,сотрудник,0,94187,автомобиль,6.962675,178030.715657,автомобиль
7,0,6,47,Высшее,0,женат / замужем,0,F,сотрудник,0,178030,профильное образование,6.962675,178030.715657,образование
8,0,41,54,Высшее,0,женат / замужем,0,F,пенсионер,0,199707,покупка жилья для сдачи,6.962675,178030.715657,недвижимость
9,0,4,26,Высшее,0,женат / замужем,0,F,сотрудник,1,143471,операции со своей недвижимостью,6.962675,178030.715657,недвижимость


**Вывод**

Провели лемматизацию для колонки `Цель кредита`, создали отдельную колонку чтобы сохранить результат. 

### Категоризация данных

Проведем категоризацию данных для колонки `Количество детей`. Полученный результат пригодится для ответа на итоговый вопрос.

In [94]:
data['children'].unique() #посмотрим на уникальные значения количества детей

array([ 1,  0,  2,  3, 20, -1,  4,  5])

Вероятно, что значение 20 ложное, предположим, что ошибка в человеческом факторе и детей на самом деле 2. Такой же вывод сделаем на счет значения -1.

In [95]:
def number_of_children(children): 
    if children == 0:
        return 'нет детей'
    if children == 1 or children == 2 or children == 20 or children == -1:
        return 'один или двое детей'
    return 'больше двух детей'
data['number_of_children'] = data['children'].apply(number_of_children)

Затем категоризируем данные для колонки `Общий доход`.

In [96]:
print("Медианное значение заработной платы:", data['total_income'].median())
def income_group(total_income):
    if total_income > data['total_income'].median():
        return 'доход выше среднего'
    return 'доход ниже среднего'
data['income_group'] = data['total_income'].apply(income_group)

Медианное значение заработной платы: 141596.0


**Вывод**

Используя категоризацию мы получили данные, с помощью которых можно ответить на основные вопросы.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [97]:
data_pivot = data.pivot_table(index='number_of_children',
                             columns='debt', values='total_income', aggfunc='count').reset_index()
data_pivot['share_of_debtors'] = data_pivot[1] / (data_pivot[1] + data_pivot[0]) * 100
data_pivot

debt,number_of_children,0,1,share_of_debtors
0,больше двух детей,349,31,8.157895
1,нет детей,13044,1063,7.535266
2,один или двое детей,6337,647,9.264032


**Вывод**

Полученные мною результаты неоднозначны. Опираясь на них, можно сказать что лучшие показатели по возврату кредитов в срок имеют клиенты без детей. А многодетные клиенты имеют чуть лучший показатель, чем клиенты с 1-2 детьми.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [98]:
data_pivot = data.pivot_table(index='family_status',
                             columns='debt', values='total_income', aggfunc='count').reset_index()
data_pivot['share of debtors'] = data_pivot[1] / (data_pivot[1] + data_pivot[0]) * 100
data_pivot

debt,family_status,0,1,share of debtors
0,Не женат / не замужем,2536,274,9.750890
1,в разводе,1110,85,7.112971
2,вдовец / вдова,896,63,6.569343
3,гражданский брак,3775,388,9.320202
4,женат / замужем,11413,931,7.542126


**Вывод**

Зависимость определенно есть. Наименьший процент должников среди категории `Вдовец/вдова`. В зарегистрированном браке процент должников заметно меньше, чем в гражданском. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [99]:
data_pivot = data.pivot_table(index='income_group',
                             columns='debt', values='total_income', aggfunc='count').reset_index()
data_pivot['share of debtors'] = data_pivot[1] / (data_pivot[1] + data_pivot[0]) * 100
data_pivot

debt,income_group,0,1,share of debtors
0,доход выше среднего,9896,839,7.815557
1,доход ниже среднего,9834,902,8.401639


**Вывод**

Зависимость так же наблюдается, но она не сильно выражена. Возможно стоило разделить уровни дохода на несколько групп и мы бы увидели более очевидную разницу.

- Как разные цели кредита влияют на его возврат в срок?

In [100]:
data_pivot = data.pivot_table(index='family_status',
                             columns='debt', values='total_income', aggfunc='count').reset_index()
data_pivot['share_of_debtors'] = data_pivot[1] / (data_pivot[1] + data_pivot[0]) * 100
data_pivot

debt,family_status,0,1,share_of_debtors
0,Не женат / не замужем,2536,274,9.750890
1,в разводе,1110,85,7.112971
2,вдовец / вдова,896,63,6.569343
3,гражданский брак,3775,388,9.320202
4,женат / замужем,11413,931,7.542126


**Вывод**

В данном случае очевидно, что наилучший показатель по возврату имеют клиенты с кредитом по `Недвижимости`. На втором месте по показателям клиенты с кредитом на `Свадьбу`. Худший показатель по возврату кредитов имеют клинты с кредитом на `Автомобиль`. 

## Шаг 4. Общий вывод

Подводя итоги отметим, что наилучшими клиентами для банка являются клиенты без детей. 

Семейное положение так же влияет на возврат кредита в срок. Люди состоящие в браке выплачивают кредит без задержек чаще. Овдовевшие заёмщики и заёмщики в разводе лидируют по своевременной выплате.

Вполне логичным оказался вывод о том, что люди с наибольшим доходом возвращают кредиты в срок чаще.

Лучшая возвращаемость для кредита, основываясь на расчетах, характерна для цели Недвижимость.
